In [1]:
from Dataset import SoccerActionDataset
from torch.utils.data import Dataset, DataLoader, random_split

In [2]:
import torch
import torch.nn as nn
from pytorchvideo.models.hub import x3d_s

In [3]:
allowed_classes = ['HEADER', 'HIGH PASS', 'OUT']

# ds = SoccerActionDataset("./data/720p/small_ds/train", "Labels-ball.json", num_frames=16, allowed_classes=allowed_classes)
ds = SoccerActionDataset("./data/720p/train", "Labels-ball.json", num_frames=16, allowed_classes=allowed_classes)

frames, label = ds[0]
print(frames.shape)

train_ratio = 0.8
train_size = int(train_ratio * len(ds))
test_size = len(ds) - train_size

train_ds, test_ds = random_split(ds, [train_size, test_size])

train_loader = DataLoader(train_ds, batch_size = 4, shuffle = True, num_workers = 4, pin_memory = True)
test_loader = DataLoader(test_ds, batch_size = 4, shuffle = True, num_workers = 4, pin_memory = True)

# train_loader = DataLoader(train_ds, batch_size = 8, shuffle = True, num_workers = 4, pin_memory = True)
# test_loader = DataLoader(test_ds, batch_size = 8, shuffle = True, num_workers = 4, pin_memory = True)

torch.Size([3, 16, 182, 182])


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
num_classes = len(ds.label_to_idx)

In [6]:
model = x3d_s(pretrained=True)

In [7]:
in_features = model.blocks[-1].proj.in_features
model.blocks[-1].proj = nn.Linear(in_features, num_classes)
model.to(device=device)

Net(
  (blocks): ModuleList(
    (0): ResNetBasicStem(
      (conv): Conv2plus1d(
        (conv_t): Conv3d(3, 24, kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), bias=False)
        (conv_xy): Conv3d(24, 24, kernel_size=(5, 1, 1), stride=(1, 1, 1), padding=(2, 0, 0), groups=24, bias=False)
      )
      (norm): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
    )
    (1): ResStage(
      (res_blocks): ModuleList(
        (0): ResBlock(
          (branch1_conv): Conv3d(24, 24, kernel_size=(1, 1, 1), stride=(1, 2, 2), bias=False)
          (branch2): BottleneckBlock(
            (conv_a): Conv3d(24, 54, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
            (norm_a): BatchNorm3d(54, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act_a): ReLU()
            (conv_b): Conv3d(54, 54, kernel_size=(3, 3, 3), stride=(1, 2, 2), padding=(1, 1, 1), groups=54, bias=False)
            (nor

In [8]:
x = torch.randn(2, 3, 16, 182, 182).to(device)
y = model(x)    
print(y.shape)  

torch.Size([2, 3])


In [9]:
from collections import Counter
label_counts = Counter(s['label'] for s in ds.samples)
label_counts

Counter({'HIGH PASS': 2748, 'HEADER': 2387, 'OUT': 1983})

In [10]:
import numpy as np
class_counts = np.array([label_counts[ds.idx_to_label[i]] for i in range(num_classes)])
class_counts

array([2387, 2748, 1983])

In [11]:
class_weights =  1 / class_counts
class_weights /= class_weights.sum() * num_classes
class_weights

array([0.10849411, 0.09424143, 0.1305978 ])

In [12]:
class_weights = torch.FloatTensor(class_weights).to(device)

In [13]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

In [14]:
from helper_scripts import *
import time

In [17]:
num_epochs = 10
best_accuracy = 0
scaler = GradScaler()
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

/tmp/ipykernel_139118/257609471.py:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [18]:
print(f"\n{'='*70}")
print(f"Starting Training")
print(f"{'='*70}")
print(f"Device: {device}")
print(f"Model: X3D-S")
print(f"Number of classes: {num_classes}")
print(f"Training samples: {len(train_ds)}")
print(f"Validation samples: {len(test_ds)}")
print(f"Batch size: 8")
print(f"Number of epochs: {num_epochs}")
print(f"{'='*70}\n")

# Training history
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': []
}

for epoch in range(num_epochs):
    start_time = time.time()
    
    print(f"\n{'='*70}")
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"{'='*70}")
    
    # Train
    train_loss, train_acc = train_one_epoch(
        model, train_loader, criterion, optimizer, scaler, device, epoch
    )
    
    # Validate
    val_loss, val_acc = validate(
        model, test_loader, criterion, device, epoch, ds.idx_to_label
    )
    
    # Update scheduler
    scheduler.step()
    
    # Save history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    # Print epoch summary
    epoch_time = time.time() - start_time
    print(f"\n{'='*70}")
    print(f"Epoch {epoch+1} Summary:")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%")
    print(f"  Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
    print(f"  Time: {epoch_time:.1f}s")
    
    # Save best model
    if val_acc > best_accuracy:
        best_accuracy = val_acc
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
            'val_loss': val_loss,
            'label_to_idx': ds.label_to_idx,
            'idx_to_label': ds.idx_to_label,
            'history': history
        }
        torch.save(checkpoint, 'best_model.pth')
        print(f"Saved best model (Val Acc: {best_accuracy:.2f}%)")
    
    print(f"{'='*70}")

print(f"\n{'='*70}")
print(f"Training Complete!")
print(f"Best Validation Accuracy: {best_accuracy:.2f}%")
print(f"{'='*70}\n")

# Save final model
torch.save({
    'model_state_dict': model.state_dict(),
    'label_to_idx': ds.label_to_idx,
    'idx_to_label': ds.idx_to_label,
    'history': history
}, 'final_model.pth')

print("Models saved:")
print("  - best_model.pth (best validation accuracy)")
print("  - final_model.pth (final epoch)")


Starting Training
Device: cuda
Model: X3D-S
Number of classes: 3
Training samples: 5694
Validation samples: 1424
Batch size: 8
Number of epochs: 10


Epoch 1/10


Epoch 1:   0%|          | 0/1424 [00:00<?, ?it/s]

Epoch 1 [Val]:   0%|          | 0/356 [00:00<?, ?it/s]

  HEADER                   :  93.0% (455/489)
  HIGH PASS                :  95.1% (504/530)
  OUT                      :  93.1% (377/405)

Epoch 1 Summary:
  Train Loss: 0.7428 | Train Acc: 80.28%
  Val Loss:   0.6161 | Val Acc:   93.82%
  Learning Rate: 0.000098
  Time: 667.8s
Saved best model (Val Acc: 93.82%)

Epoch 2/10


Epoch 2:   0%|          | 0/1424 [00:00<?, ?it/s]

...

Epoch 9 [Val]:   0%|          | 0/356 [00:00<?, ?it/s]

  HEADER                   :  99.6% (487/489)
  HIGH PASS                :  98.3% (521/530)
  OUT                      : 100.0% (405/405)

Epoch 9 Summary:
  Train Loss: 0.5547 | Train Acc: 99.68%
  Val Loss:   0.5585 | Val Acc:   99.23%
  Learning Rate: 0.000002
  Time: 687.4s

Epoch 10/10


Epoch 10:   0%|          | 0/1424 [00:00<?, ?it/s]

Epoch 10 [Val]:   0%|          | 0/356 [00:00<?, ?it/s]

  HEADER                   :  99.6% (487/489)
  HIGH PASS                :  98.3% (521/530)
  OUT                      : 100.0% (405/405)

Epoch 10 Summary:
  Train Loss: 0.5543 | Train Acc: 99.70%
  Val Loss:   0.5584 | Val Acc:   99.23%
  Learning Rate: 0.000000
  Time: 629.4s

Training Complete!
Best Validation Accuracy: 99.23%

Models saved:
  - best_model.pth (best validation accuracy)
  - final_model.pth (final epoch)
